In [12]:
!pip install easyocr opencv-python-headless gradio

In [11]:
# ✅ Imports
import os
import cv2
import numpy as np
import gradio as gr
from pathlib import Path
from google.colab import drive
import matplotlib.pyplot as plt
import easyocr
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# ✅ Mount Google Drive
drive.mount('/content/drive')

# ✅ Constants
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 5
DATASET_PATH = '/content/drive/My Drive/Indian_Number_Plates'  # Make sure this is your correct dataset path

data = []
plates = []

reader = easyocr.Reader(['en'])

# ✅ Load and process number plate images
print("Loading vehicle images and extracting number plates...")

# Recursively collect image files
image_paths = list(Path(DATASET_PATH).rglob("*.jpg")) + list(Path(DATASET_PATH).rglob("*.png"))

print(f"Found {len(image_paths)} images")

for img_path in image_paths:
    try:
        image = cv2.imread(str(img_path))
        if image is None:
            print(f"Skipped {img_path} (not a valid image)")
            continue

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blur = cv2.bilateralFilter(gray, 11, 17, 17)
        edged = cv2.Canny(blur, 30, 200)

        contours, _ = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

        plate = None
        for c in contours:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.018 * peri, True)
            if len(approx) == 4:
                x, y, w, h = cv2.boundingRect(approx)
                plate = image[y:y + h, x:x + w]
                break

        if plate is not None:
            plate_resized = cv2.resize(plate, (IMG_SIZE, IMG_SIZE))
            data.append(plate_resized)
            result = reader.readtext(plate)
            if result:
                text = result[0][1]
            else:
                text = "unknown"
            plates.append(text)

    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        continue

# ✅ Preprocessing
if len(data) == 0:
    raise ValueError("❌ No valid images found. Please check your dataset path and image formats.")

data = np.array(data) / 255.0
plates = np.array(plates)

# ✅ Train-test split (optional, not used by OCR model)
X_train, X_test, y_train, y_test = train_test_split(data, plates, test_size=0.2, random_state=42)

# ✅ Prediction function using EasyOCR (no training needed)
def predict_plate(image):
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.bilateralFilter(gray, 11, 17, 17)
    edged = cv2.Canny(blur, 30, 200)

    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    for c in contours:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.018 * peri, True)
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            plate = image[y:y + h, x:x + w]
            result = reader.readtext(plate)
            if result:
                return result[0][1]
            else:
                return "Text not recognized"
    return "Number plate not found"

# ✅ Gradio interface
interface = gr.Interface(
    fn=predict_plate,
    inputs=gr.Image(type='pil'),
    outputs='text',
    title="Indian Vehicle Number Plate Detector",
    description="Upload a vehicle image to extract the number plate text using EasyOCR."
)

interface.launch(share=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading vehicle images and extracting number plates...
Found 27 images
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a5d28171a7cb76d62d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
